# Pytorch tutorial for image classification
Checkout the Pytorch tutorial in this [link](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [44]:
import torch
import torchvision
import torchvision.transforms as transforms

## Loading the Data (CIFAR10)
Downloading the data, normalize it and transform to tensors, and wrap it in DataLoaders for 'train' and 'test'.

In [45]:
from ToGrowABackbone.training_utils import WeightedDataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
classes_weights=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
trainset = WeightedDataset(trainset, classes_weights)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## The Model
Define a neural network model.
The model is defined as a class which inherits from the pytorch class `Model`.
Notice that first, we define the layers in the `__init__` and only after we use the defined layers in the `forward`.
This writing is similar for tensorflow as well.

In [51]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

We define the optimizer and the loss function, which defines how to train our model:

In [52]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training
The training process:

In [53]:
from ToGrowABackbone.training_utils import evaluate

for epoch in range(6):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch}:')
    print(f'probabilities: {trainloader.dataset.probabilities}')
    evaluate(net, testloader, classes)
    classes_weights[3] += 0.5
    trainset.change_weights(classes_weights)

print('Finished Training')

Epoch 0:
probabilities: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Accuracy for class plane is: 29.2 %
Accuracy for class car   is: 39.2 %
Accuracy for class bird  is: 11.3 %
Accuracy for class cat   is: 0.0 %
Accuracy for class deer  is: 17.4 %
Accuracy for class dog   is: 55.8 %
Accuracy for class frog  is: 0.0 %
Accuracy for class horse is: 0.0 %
Accuracy for class ship  is: 28.8 %
Accuracy for class truck is: 36.0 %
Epoch 1:
probabilities: [0.0952381  0.0952381  0.0952381  0.14285714 0.0952381  0.0952381
 0.0952381  0.0952381  0.0952381  0.0952381 ]
Accuracy for class plane is: 52.4 %
Accuracy for class car   is: 42.8 %
Accuracy for class bird  is: 0.2 %
Accuracy for class cat   is: 73.5 %
Accuracy for class deer  is: 0.0 %
Accuracy for class dog   is: 0.0 %
Accuracy for class frog  is: 17.6 %
Accuracy for class horse is: 28.2 %
Accuracy for class ship  is: 32.5 %
Accuracy for class truck is: 24.0 %
Epoch 2:
probabilities: [0.09090909 0.09090909 0.09090909 0.18181818 0.09090909 0.09

# Evaluation
Evaluate our model on the test set.

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))